In [1]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=89813a716aded0c2eeab244b4bee71b65579a45cf93fa6fcb4c971b670315d2b
  Stored in directory: /root/.cach

In [2]:
from flair.data import Sentence
from flair.models import SequenceTagger

In [3]:
# load tagger
tagger = SequenceTagger.load("flair/pos-english")

# make example sentence
sentence = Sentence("I love Berlin.")

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('pos'):
    print(entity)

pytorch_model.bin:   0%|          | 0.00/249M [00:00<?, ?B/s]

2024-11-24 12:34:11,737 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
Sentence[4]: "I love Berlin." → ["I"/PRP, "love"/VBP, "Berlin"/NNP, "."/.]
The following NER tags are found:


### Initial pipeline:

In [4]:
!pip install conllu

In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    XLMRobertaTokenizerFast,  # Changed from XLMRobertaTokenizer to XLMRobertaTokenizerFast
    XLMRobertaForTokenClassification,
    AdamW,
    get_linear_schedule_with_warmup
)
from datasets import load_dataset
import numpy as np
from sklearn.metrics import classification_report
import pandas as pd
from typing import List, Dict, Tuple
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [7]:
class POSDataset(Dataset):
    def __init__(self, texts: List[List[str]], tags: List[List[str]],
                 tokenizer, tag2id: Dict[str, int], max_len: int = 128):
        self.texts = texts
        self.tags = tags
        self.tokenizer = tokenizer
        self.tag2id = tag2id
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        words = self.texts[idx]
        tags = self.tags[idx]

        encoded = self.tokenizer(
            words,
            is_split_into_words=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        label_ids = []
        word_ids = encoded.word_ids()

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(self.tag2id[tags[word_idx]])

        return {
            'input_ids': encoded['input_ids'].squeeze(),
            'attention_mask': encoded['attention_mask'].squeeze(),
            'labels': torch.tensor(label_ids)
        }

In [18]:
class POSTaggingPipeline:
    def __init__(self, model_name: str = "xlm-roberta-base"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logger.info(f"Using device: {self.device}")

        self.model_name = model_name
        self.tokenizer = XLMRobertaTokenizerFast.from_pretrained(model_name)
        logger.info("Initialized Fast tokenizer")

        self.tag2id = None
        self.id2tag = None
        self.model = None

    def prepare_data(self, dataset_name: str, split: str = "train") -> Tuple[List[List[str]], List[List[str]]]:
        """
        Load and prepare data from Universal Dependencies dataset.

        Args:
            dataset_name (str): Name of the UD dataset (e.g., 'en_ewt', 'wo_wtb')
            split (str): Dataset split ('train', 'validation', 'test')

        Returns:
            Tuple[List[List[str]], List[List[str]]]: Tuple of (texts, tags)
        """
        logger.info(f"Loading {dataset_name} dataset, {split} split")
        try:
            # Load the dataset
            dataset = load_dataset("universal_dependencies", dataset_name)

            # Get the specified split
            data_split = dataset[split]

            # Extract texts and tags
            texts = [item['tokens'] for item in data_split]
            tags = [item['upos'] for item in data_split]

            logger.info(f"Loaded {len(texts)} sentences from {dataset_name} {split} split")

            # Basic validation
            assert all(len(text) == len(tag) for text, tag in zip(texts, tags)), \
                "Mismatch between text and tag lengths"

            return texts, tags

        except Exception as e:
            logger.error(f"Error loading dataset {dataset_name}: {str(e)}")
            raise

    def initialize_model(self, num_labels: int):
        """Initialize the model with proper classification head"""
        logger.info(f"Initializing model with {num_labels} labels")

        self.model = XLMRobertaForTokenClassification.from_pretrained(
            self.model_name,
            num_labels=num_labels
        )

        # Initialize classification layer
        torch.nn.init.xavier_uniform_(self.model.classifier.weight)
        torch.nn.init.zeros_(self.model.classifier.bias)

        self.model = self.model.to(self.device)
        logger.info("Model initialized and moved to device")

    def create_tag_mappings(self, tags: List[List[str]]):
        """Create tag to ID mappings"""
        unique_tags = sorted(list(set(tag for seq in tags for tag in seq)))
        self.tag2id = {tag: i for i, tag in enumerate(unique_tags)}
        self.id2tag = {i: tag for tag, i in self.tag2id.items()}
        logger.info(f"Created mappings for {len(unique_tags)} unique tags: {unique_tags}")

    def train(self, train_texts: List[List[str]], train_tags: List[List[str]],
              eval_texts: List[List[str]] = None, eval_tags: List[List[str]] = None,
              epochs: int = 3, batch_size: int = 16, learning_rate: float = 2e-5):
        """Train the model on source language data"""
        if self.tag2id is None:
            self.create_tag_mappings(train_tags)

        self.initialize_model(len(self.tag2id))

        train_dataset = POSDataset(train_texts, train_tags, self.tokenizer, self.tag2id)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        if eval_texts:
            eval_dataset = POSDataset(eval_texts, eval_tags, self.tokenizer, self.tag2id)
            eval_loader = DataLoader(eval_dataset, batch_size=batch_size)

        optimizer = AdamW(self.model.parameters(), lr=learning_rate, weight_decay=0.01)
        total_steps = len(train_loader) * epochs
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=total_steps // 10,
            num_training_steps=total_steps
        )

        logger.info("Starting training...")
        best_f1 = 0
        for epoch in range(epochs):
            self.model.train()
            total_loss = 0

            for batch_idx, batch in enumerate(train_loader):
                optimizer.zero_grad()

                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

                loss = outputs.loss
                total_loss += loss.item()

                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                optimizer.step()
                scheduler.step()

                if (batch_idx + 1) % 100 == 0:
                    logger.info(f"Epoch {epoch+1}/{epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

            avg_loss = total_loss / len(train_loader)
            logger.info(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")

            if eval_texts:
                metrics = self.evaluate(eval_loader)
                current_f1 = metrics['weighted avg']['f1-score']
                logger.info(f"Validation F1: {current_f1:.4f}")

                if current_f1 > best_f1:
                    best_f1 = current_f1
                    logger.info(f"New best F1 score: {best_f1:.4f}")

    def evaluate(self, eval_loader: DataLoader) -> Dict:
        """Evaluate the model"""
        self.model.eval()
        true_labels = []
        pred_labels = []

        logger.info("Starting evaluation...")
        with torch.no_grad():
            for batch in eval_loader:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['labels']

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )

                predictions = torch.argmax(outputs.logits, dim=2)

                for i in range(labels.shape[0]):
                    true_seq = [self.id2tag[l.item()] for l in labels[i] if l.item() != -100]
                    pred_seq = [self.id2tag[p.item()] for p, l in zip(predictions[i], labels[i]) if l.item() != -100]
                    true_labels.extend(true_seq)
                    pred_labels.extend(pred_seq)

        return classification_report(true_labels, pred_labels, output_dict=True)

    def predict(self, texts: List[List[str]]) -> List[List[str]]:
        """
        Predict POS tags for new texts while maintaining original token alignment
        """
        if self.model is None:
            raise ValueError("Model has not been trained yet!")

        self.model.eval()
        predictions = []

        logger.info("Starting prediction...")
        with torch.no_grad():
            for text in texts:
                # Tokenize the text while keeping track of word IDs
                encoded = self.tokenizer(
                    text,
                    is_split_into_words=True,
                    return_tensors='pt',
                    padding=True,
                    truncation=True
                )

                input_ids = encoded['input_ids'].to(self.device)
                attention_mask = encoded['attention_mask'].to(self.device)
                word_ids = encoded.word_ids()

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )

                preds = torch.argmax(outputs.logits, dim=2)[0].cpu().numpy()

                # Initialize predictions for this sentence
                sent_predictions = []
                prev_word_idx = None

                # Align predictions with original tokens
                for token_idx, word_idx in enumerate(word_ids):
                    if word_idx is None:
                        continue
                    if word_idx != prev_word_idx:
                        sent_predictions.append(self.id2tag[preds[token_idx]])
                        prev_word_idx = word_idx

                # Ensure predictions match the input length
                sent_predictions = sent_predictions[:len(text)]

                # If predictions are shorter (due to truncation), pad with the most common tag
                if len(sent_predictions) < len(text):
                    most_common_tag = max(set(self.id2tag.values()), key=list(self.id2tag.values()).count)
                    sent_predictions.extend([most_common_tag] * (len(text) - len(sent_predictions)))

                predictions.append(sent_predictions)

        logger.info(f"Generated predictions for {len(predictions)} sentences")

        # Verify prediction lengths match input lengths
        for text, pred in zip(texts, predictions):
            assert len(text) == len(pred), f"Mismatch in lengths: text={len(text)}, pred={len(pred)}"

        return predictions

In [ ]:
"""
class POSTaggingPipeline:
    def __init__(self, model_name: str = "xlm-roberta-base"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logger.info(f"Using device: {self.device}")

        self.model_name = model_name
        self.tokenizer = XLMRobertaTokenizerFast.from_pretrained(model_name)
        logger.info("Initialized Fast tokenizer")

        self.tag2id = None
        self.id2tag = None
        self.model = None

    def prepare_data(self, dataset_name: str, split: str = "train") -> Tuple[List[List[str]], List[List[str]]]:
        """
        Load and prepare data from Universal Dependencies dataset.

        Args:
            dataset_name (str): Name of the UD dataset (e.g., 'en_ewt', 'wo_wtb')
            split (str): Dataset split ('train', 'validation', 'test')

        Returns:
            Tuple[List[List[str]], List[List[str]]]: Tuple of (texts, tags)
        """
        logger.info(f"Loading {dataset_name} dataset, {split} split")
        try:
            # Load the dataset
            dataset = load_dataset("universal_dependencies", dataset_name)

            # Get the specified split
            data_split = dataset[split]

            # Extract texts and tags
            texts = [item['tokens'] for item in data_split]
            tags = [item['upos'] for item in data_split]

            logger.info(f"Loaded {len(texts)} sentences from {dataset_name} {split} split")

            # Basic validation
            assert all(len(text) == len(tag) for text, tag in zip(texts, tags)), \
                "Mismatch between text and tag lengths"

            return texts, tags

        except Exception as e:
            logger.error(f"Error loading dataset {dataset_name}: {str(e)}")
            raise

    def initialize_model(self, num_labels: int):
        """Initialize the model with proper classification head"""
        logger.info(f"Initializing model with {num_labels} labels")

        self.model = XLMRobertaForTokenClassification.from_pretrained(
            self.model_name,
            num_labels=num_labels
        )

        # Initialize classification layer
        torch.nn.init.xavier_uniform_(self.model.classifier.weight)
        torch.nn.init.zeros_(self.model.classifier.bias)

        self.model = self.model.to(self.device)
        logger.info("Model initialized and moved to device")

    def create_tag_mappings(self, tags: List[List[str]]):
        """Create tag to ID mappings"""
        unique_tags = sorted(list(set(tag for seq in tags for tag in seq)))
        self.tag2id = {tag: i for i, tag in enumerate(unique_tags)}
        self.id2tag = {i: tag for tag, i in self.tag2id.items()}
        logger.info(f"Created mappings for {len(unique_tags)} unique tags: {unique_tags}")

    def train(self, train_texts: List[List[str]], train_tags: List[List[str]],
              eval_texts: List[List[str]] = None, eval_tags: List[List[str]] = None,
              epochs: int = 3, batch_size: int = 16, learning_rate: float = 2e-5):
        """Train the model on source language data"""
        if self.tag2id is None:
            self.create_tag_mappings(train_tags)

        self.initialize_model(len(self.tag2id))

        train_dataset = POSDataset(train_texts, train_tags, self.tokenizer, self.tag2id)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        if eval_texts:
            eval_dataset = POSDataset(eval_texts, eval_tags, self.tokenizer, self.tag2id)
            eval_loader = DataLoader(eval_dataset, batch_size=batch_size)

        optimizer = AdamW(self.model.parameters(), lr=learning_rate, weight_decay=0.01)
        total_steps = len(train_loader) * epochs
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=total_steps // 10,
            num_training_steps=total_steps
        )

        logger.info("Starting training...")
        best_f1 = 0
        for epoch in range(epochs):
            self.model.train()
            total_loss = 0

            for batch_idx, batch in enumerate(train_loader):
                optimizer.zero_grad()

                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['labels'].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

                loss = outputs.loss
                total_loss += loss.item()

                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                optimizer.step()
                scheduler.step()

                if (batch_idx + 1) % 100 == 0:
                    logger.info(f"Epoch {epoch+1}/{epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

            avg_loss = total_loss / len(train_loader)
            logger.info(f"Epoch {epoch+1}/{epochs}, Average Loss: {avg_loss:.4f}")

            if eval_texts:
                metrics = self.evaluate(eval_loader)
                current_f1 = metrics['weighted avg']['f1-score']
                logger.info(f"Validation F1: {current_f1:.4f}")

                if current_f1 > best_f1:
                    best_f1 = current_f1
                    logger.info(f"New best F1 score: {best_f1:.4f}")

    def evaluate(self, eval_loader: DataLoader) -> Dict:
        """Evaluate the model"""
        self.model.eval()
        true_labels = []
        pred_labels = []

        logger.info("Starting evaluation...")
        with torch.no_grad():
            for batch in eval_loader:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels = batch['labels']

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )

                predictions = torch.argmax(outputs.logits, dim=2)

                for i in range(labels.shape[0]):
                    true_seq = [self.id2tag[l.item()] for l in labels[i] if l.item() != -100]
                    pred_seq = [self.id2tag[p.item()] for p, l in zip(predictions[i], labels[i]) if l.item() != -100]
                    true_labels.extend(true_seq)
                    pred_labels.extend(pred_seq)

        return classification_report(true_labels, pred_labels, output_dict=True)

    def predict(self, texts: List[List[str]]) -> List[List[str]]:
        """Predict POS tags for new texts"""
        if self.model is None:
            raise ValueError("Model has not been trained yet!")

        self.model.eval()
        dataset = POSDataset(texts, [[self.id2tag[0]]*len(text) for text in texts],
                           self.tokenizer, self.tag2id)
        loader = DataLoader(dataset, batch_size=1)
        predictions = []

        logger.info("Starting prediction...")
        with torch.no_grad():
            for batch in loader:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                )

                preds = torch.argmax(outputs.logits, dim=2)
                pred_tags = [self.id2tag[p.item()] for p, m in zip(preds[0], batch['attention_mask'][0])
                           if m.item() == 1][1:-1]  # Remove CLS and SEP
                predictions.append(pred_tags)

        logger.info(f"Generated predictions for {len(predictions)} sentences")
        return predictions
"""

In [19]:
# Initialize pipeline
pipeline = POSTaggingPipeline()

In [20]:
# Load English data
en_train_texts, en_train_tags = pipeline.prepare_data("en_ewt", "train")
en_eval_texts, en_eval_tags = pipeline.prepare_data("en_ewt", "validation")

In [11]:
import random

In [12]:
# Adjust the dataset size by sampling a smaller fraction for quicker training
'''
en_train_texts_reduced = random.sample(en_train_texts, int(len(en_train_texts) * 0.1))
en_train_tags_reduced = random.sample(en_train_tags, int(len(en_train_tags) * 0.1))
en_eval_texts_reduced = random.sample(en_eval_texts, int(len(en_eval_texts) * 0.1))
en_eval_tags_reduced = random.sample(en_eval_tags, int(len(en_eval_tags) * 0.1))
'''

'\nen_train_texts_reduced = random.sample(en_train_texts, int(len(en_train_texts) * 0.1))\nen_train_tags_reduced = random.sample(en_train_tags, int(len(en_train_tags) * 0.1))\nen_eval_texts_reduced = random.sample(en_eval_texts, int(len(en_eval_texts) * 0.1))\nen_eval_tags_reduced = random.sample(en_eval_tags, int(len(en_eval_tags) * 0.1))\n'

In [21]:
# Adjust dataset size before training
# Assuming `train_texts` and `train_tags` are lists of texts and tags respectively
def reduce_dataset_size(texts, tags, fraction=0.1, random_seed=42):
    random.seed(random_seed)

    # Pair texts with their corresponding tags for sampling
    paired_data = list(zip(texts, tags))

    # Sample a fraction of the paired data
    reduced_data = random.sample(paired_data, int(len(paired_data) * fraction))

    # Unzip the reduced data back into separate lists
    reduced_texts, reduced_tags = zip(*reduced_data)
    return list(reduced_texts), list(reduced_tags)

# Reduce dataset size for training and evaluation
en_train_texts_reduced, en_train_tags_reduced = reduce_dataset_size(en_train_texts, en_train_tags, fraction=0.1)
en_eval_texts_reduced, en_eval_tags_reduced = reduce_dataset_size(en_eval_texts, en_eval_tags, fraction=0.1)

In [22]:
# Train model
pipeline.train(
    train_texts=en_train_texts_reduced,
    train_tags=en_train_tags_reduced,
    eval_texts=en_eval_texts_reduced,
    eval_tags=en_eval_tags_reduced,
    epochs=1
)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531

In [15]:
# Load Wolof test data
wo_texts, wo_tags = pipeline.prepare_data("wo_wtb", "test")

Generating train split:   0%|          | 0/1188 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/449 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/470 [00:00<?, ? examples/s]

In [23]:
# Get predictions
predictions = pipeline.predict(wo_texts)

In [34]:
# Calculate metrics
from sklearn.metrics import classification_report
print(classification_report(
    [tag for sent in wo_tags for tag in sent],
    [tag for sent in predictions for tag in sent]
))

              precision    recall  f1-score   support

           0       0.28      0.11      0.15      1744
           1       0.59      1.00      0.74      1197
           2       0.00      0.00      0.00       765
           3       0.06      0.10      0.07       142
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00       212
           6       0.00      0.00      0.00         2
           7       0.01      0.07      0.02       145
           8       0.01      0.00      0.00       804
           9       0.25      0.13      0.17       265
          10       0.15      0.92      0.25       706
          11       0.61      0.13      0.22      1400
          12       0.00      0.00      0.00         0
          13       0.04      0.02      0.02       309
          14       0.00      0.00      0.00       303
          15       0.00      0.00      0.00         3
          16       0.32      0.13      0.19      1752
          17       0.15    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_